In [2]:
snakemake.__dict__

In [3]:
from pathlib import Path
Path("./workflow/config/" + snakemake.config["viz"]["stylefile"]).resolve()

In [4]:
import pickle
import pandas as pd
import numpy as np
#%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

p = '/home/paulina/Documents/github/InsectNavigation-RL/v2/inverse/workflow/config/trajectory.mplstyle'
plt.style.use(p)

from cycler import cycler
import seaborn as sns
from scipy import interpolate

scale = 1


def draw_trajectory(df, ax):
    x = df.path_x / scale
    y = df.path_y / scale
    ax.plot(x, y, alpha=0.75)
    return ax

    
def df_interpolate(df, max_=5):
    def per_df(d):
        x = np.array(d['path_x'])
        y = np.array(d['path_y'])

        # remove two consecutive equal points for interpolation
        okay = np.where(np.abs(np.diff(x)) + np.abs(np.diff(y)) > 0)
        x = np.r_[x[okay], x[-1]]
        y = np.r_[y[okay], y[-1]]

        tck,u=interpolate.splprep([x,y],s=0.0)
        x_i,y_i= interpolate.splev(np.linspace(0,1,(len(x)*10)),tck)

        new_rows = d.iloc[-1].to_dict()
        new_rows.update({'path_x' : x_i, 'path_y': y_i})
        return pd.DataFrame(new_rows)
    dfs = []
    agents = set(df["ant_nb"])
    trials = set(df["trial_nb"])
    if len(trials) > max_:
        trials = [arr[0] for arr in np.array_split(list(trials), max_)]
    for agent in agents:
        for trial in trials:
            dfs.append(per_df(df[(df["ant_nb"]==agent) & (df["trial_nb"]==trial)]))
    return pd.concat(dfs, ignore_index=True)


In [5]:
%%capture
raw, discrete, interpolated = snakemake.input # be sure this is correct
df_raw = pd.read_csv(raw)
df_discrete = pd.read_csv(discrete)
df_interpolated = pd.read_csv(interpolated)

In [6]:
#%matplotlib inline
fig, ax = plt.subplots(1, 1, figsize=(10 ,10))
for ant in [32]:#pd.unique(df["ant_nb"]):
    ant_df = df_interpolated[df_interpolated.ant_nb == ant]

    for run in pd.unique(ant_df["trial_nb"]):

        traj = ant_df[ant_df.trial_nb == run]
        draw_trajectory(traj, ax)

plt.show()

In [7]:
import matplotlib.ticker as ticker


def draw_trajectories(df, ants, axes=None, do_interpolation=False):
    
    if axes is None:
        fig, axes = plt.subplots(max(1,len(ants) // 2),min(len(ants),2))
    
    if do_interpolation:
        df = df_interpolate(df)
        
    for ant, ax in zip(ants, fig.axes):#pd.unique(df["ant_nb"]):
        ant_df = df[df.ant_nb == ant]
        runs = pd.unique(ant_df["trial_nb"])
        spacing = np.linspace(0,1,len(runs))
        colors = list(reversed([(x,x,0,1-x/3) for x in spacing]))        

        for run, color in zip(runs, colors):

            traj = ant_df[ant_df.trial_nb == run]
            
            hello = traj.plot(x="path_x", y="path_y", ax=ax, label=run, c=color)
            traj.plot.scatter(x="path_x", y="path_y", ax=ax, color=color)
        

        ax.axis('equal')
        ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
        ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
        
        ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(1))
        
        ax.set_xlim(left=0)
        
        
        #ax.set_aspect('equal')

        ax.grid(which='both')
            
            #traj.rolling(3).mean().plot(x="path_x", y="path_y", ax=ax, c=color, label="mean")

In [8]:
draw_trajectories(df_raw, [32,33])
#fig.tight_layout()
plt.savefig(snakemake.output[0])

In [9]:
draw_trajectories(df_discrete, [32,33])
plt.savefig(snakemake.output[1])

In [10]:
draw_trajectories(df_interpolated, [32,33])
plt.savefig(snakemake.output[2])

In [11]:
df_discrete

In [12]:
df_interpolated

In [13]:
#draw_trajectories(df_interpolate(df_interpolated), [32,33])

In [66]:
def draw_bins(df, bins:tuple[int,int] =None):
    if bins is None:
        bins = (
                len(pd.unique(df.path_x)),
                len(pd.unique(df.path_x)) * int((df_raw.path_y.max() // df_raw.path_x.max()))    
               )
    print(bins)
    
    heatmap, xedges, yedges = np.histogram2d(df.path_x, df.path_y, bins=bins)

    plt.figure()#figsize = (10,10))
    plt.clf()
    plt.imshow(heatmap.T, origin='lower', aspect='equal')

In [75]:
d = df_raw # df_raw[df_raw.ant_nb==33]
#bins = (10, int(10 * (d.path_y.max() / d.path_x.max())))
draw_bins(d, bins=(22,58))
plt.savefig(snakemake.output[3])

In [74]:
for ant in [32]:#,33,34]:#pd.unique(df_interpolated["ant_nb"]):
    ant_df = df_interpolated[df_interpolated.ant_nb == ant]
    
draw_bins(df_interpolated, bins=(22,58))
plt.savefig(snakemake.output[4])

In [60]:
from collections import Counter
from matplotlib import cm as CM


fig, ax = plt.subplots(1, 1, figsize=(10 ,13))

x = []
y = []
df = df_interpolated
#for ant in pd.unique(df["ant_nb"]):
for ant in [32]:
    ant_df = df[df.ant_nb == ant]
    for run in pd.unique(ant_df["trial_nb"]):
        traj = ant_df[ant_df.trial_nb == run]
        
        draw_trajectory(df_interpolate(traj), ax)
        x.extend(list(traj['path_x']))
        y.extend(list(traj['path_y']))

a = list(zip(x,y))
frequency_list = Counter(a)
z = [frequency_list[tup] for tup in a]

gridsize=22

extent = [np.array(x).min(), np.array(x).max(), np.array(y).min(), np.array(y).max()]
plt.hexbin(x, y, gridsize=gridsize, cmap=CM.gnuplot2, bins=None, extent=extent)
plt.axis(extent)
#cb = PLT.colorbar()

plt.savefig(snakemake.output[5])

In [85]:
fig, ax = plt.subplots(1, 1, figsize=(10 ,13))

x = []
y = []
df = df_interpolated

for ant in [32]:#pd.unique(df["ant_nb"]):
    ant_df = df[df.ant_nb == ant]

    for run in pd.unique(ant_df["trial_nb"]):

        traj = ant_df[ant_df.trial_nb == run].copy(deep=True)
        
        traj.path_x = traj['path_x'] - traj.path_x.min()
        traj.path_y = traj['path_y'] - traj.path_y.min()
        
        draw_trajectory(df_interpolate(traj), ax)
        x.extend(list(traj['path_x']))
        y.extend(list(traj['path_y']))
        

heatmap, xedges, yedges = np.histogram2d(x, y, bins=(16,51))
plt.imshow(heatmap.T, origin='lower')
plt.savefig(snakemake.output[6])